In [2]:
import numpy as np
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st
import sys
import os
import numpy as np
sys.path.append('../quantnn-keras/layers/')

In [3]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.losses import squared_hinge
from keras.layers import Flatten, Dense, Activation, BatchNormalization, Conv1D
from keras.callbacks import ReduceLROnPlateau, TensorBoard

from keras.models import Sequential, Model
import numpy as np

from binary_layers import BinaryConv2D, BinaryConv1D
from binary_ops import binary_tanh as binary_tanh_op
from keras.regularizers import l2

Using TensorFlow backend.


In [4]:
def A(sample):
    feat = []
    for col in range(0,sample.shape[1]):
        average = np.average(sample[:,col])
        feat.append(average)

    return feat

def SD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        std = np.std(sample[:, col])
        feat.append(std)

    return feat

def AAD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        add = np.mean(np.absolute(data - np.mean(data)))
        feat.append(add)

    return feat

def ARA(sample):
    #Average Resultant Acceleration[1]:
    # Average of the square roots of the sum of the values of each axis squared √(xi^2 + yi^2+ zi^2) over the ED
    feat = []
    sum_square = 0
    sample = np.power(sample, 2)
    for col in range(0, sample.shape[1]):
        sum_square = sum_square + sample[:, col]

    sample = np.sqrt(sum_square)
    average = np.average(sample)
    feat.append(average)
    return feat

def TBP(sample):
    from scipy import signal
    feat = []
    sum_of_time = 0
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        peaks = signal.find_peaks_cwt(data, np.arange(1,4))

        feat.append(peaks)

    return feat

def feature_extraction(X):
    #Extracts the features, as mentioned by Catal et al. 2015
    # Average - A,
    # Standard Deviation - SD,
    # Average Absolute Difference - AAD,
    # Average Resultant Acceleration - ARA(1),
    # Time Between Peaks - TBP
    X_tmp = []
    for sample in X:
        features = A(sample)
        features = np.hstack((features, A(sample)))
        features = np.hstack((features, SD(sample)))
        features = np.hstack((features, AAD(sample)))
        features = np.hstack((features, ARA(sample)))
        #features = np.hstack((features, TBP(sample)))
        X_tmp.append(features)

    X = np.array(X_tmp)
    return X

In [5]:
callbacks = [ReduceLROnPlateau('val_loss', patience=5, verbose=1), TensorBoard()]

In [ ]:
data_input_file = 'data/LOSO/MHEALTH.npz'

tmp = np.load(data_input_file)
X = tmp['X']
y = tmp['y']
folds = tmp['folds']

n_class = y.shape[1]

avg_acc = []
avg_recall = []
avg_f1 = []
y = np.argmax(y, axis=1)

print('ConvNet Template 2017 {}'.format(data_input_file))

for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]

    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]
    
    X_train = feature_extraction(X_train)
    X_test = feature_extraction(X_test)
    
    
    X_train = X_train.reshape(-1,1001,1)
    X_test = X_test.reshape(-1,1001,1)
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    # Create Model
    model = Sequential()

    _momentum = 0.1
    _epsilon = 0.001
    
    model.add(
        BinaryConv1D(
            kernel_size=10,
            filters=64,
            padding='same',
            input_shape=(1001,1),
            kernel_lr_multiplier=10,use_bias=True))
    model.add(BatchNormalization(momentum=_momentum, epsilon=_epsilon))
    model.add(Activation(binary_tanh_op))


    model.add(
        BinaryConv1D(
            kernel_size=10, filters=128, padding='same',
            kernel_lr_multiplier=10,use_bias=True))
    model.add(BatchNormalization(momentum=_momentum, epsilon=_epsilon))
    model.add(Activation(binary_tanh_op))


    model.add(
        BinaryConv1D(
            kernel_size=10, filters=128, padding='same',
            kernel_lr_multiplier=10,use_bias=True))
    model.add(BatchNormalization(momentum=_momentum, epsilon=_epsilon))
    model.add(Activation(binary_tanh_op))

    model.add(Flatten())
    model.add(Dense(12, use_bias=True))
    model.add(BatchNormalization(momentum=_momentum, epsilon=_epsilon))

    # Train
    adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    #sgd = SGD(lr=0.001)
    model.compile(loss=squared_hinge, optimizer=adam, metrics=['accuracy'])
    #model.summary()

    model.fit(
        X_train, y_train, batch_size=50, epochs=100, validation_data=(X_test,y_test), verbose=0 )
    print (model.evaluate(X_test,y_test))

In [ ]:
print (X_train.shape, X_test.shape)
print (y_train.shape, y_test.shape)

In [ ]:
X_train = X_train.reshape(-1,1)
X_test = X_test.reshape(-1,1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
print (X_train.shape, X_test.shape)
print (y_train.shape, y_test.shape)

In [ ]:
print (folds.shape, len(folds))

In [6]:
data_input_file = 'data/LOSO/MHEALTH.npz'

tmp = np.load(data_input_file)
X = tmp['X']
y = tmp['y']
folds = tmp['folds']

n_class = y.shape[1]

avg_acc = []
avg_recall = []
avg_f1 = []
y = np.argmax(y, axis=1)

In [7]:
i = 0

train_idx = folds[i][0]
test_idx = folds[i][1]

X_train, y_train = X[train_idx], y[train_idx]
X_test, y_test = X[test_idx], y[test_idx]

X_train = feature_extraction(X_train)
X_test = feature_extraction(X_test)



X_train = X_train.reshape(-1,1001,1)
X_test = X_test.reshape(-1,1001,1)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Create Model
model = Sequential()

_momentum = 0.1
_epsilon = 0.0001

model.add(
    BinaryConv1D(
        kernel_size=10,
        filters=64,
        padding='same',
        input_shape=(1001,1),
        kernel_lr_multiplier=10,use_bias=True))
model.add(BatchNormalization(momentum=_momentum, epsilon=_epsilon))
model.add(Activation(binary_tanh_op))


model.add(
    BinaryConv1D(
        kernel_size=10, filters=128, padding='same',
        kernel_lr_multiplier=10,use_bias=True))
model.add(BatchNormalization(momentum=_momentum, epsilon=_epsilon))
model.add(Activation(binary_tanh_op))


model.add(
    BinaryConv1D(
        kernel_size=10, filters=128, padding='same',
        kernel_lr_multiplier=10,use_bias=True))
model.add(BatchNormalization(momentum=_momentum, epsilon=_epsilon))
model.add(Activation(binary_tanh_op))

model.add(Flatten())
model.add(Dense(12, use_bias=True))
model.add(BatchNormalization(momentum=_momentum, epsilon=_epsilon))

# Train
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#sgd = SGD(lr=0.001)
model.compile(loss=squared_hinge, optimizer=adam, metrics=['accuracy'])
#model.summary()

model.fit(
    X_train, y_train, batch_size=128, epochs=20, validation_data=(X_test,y_test), verbose=1, )
print (model.evaluate(X_test,y_test))

KeyboardInterrupt: 

In [7]:
i = 0

train_idx = folds[i][0]
test_idx = folds[i][1]

X_train, y_train = X[train_idx], y[train_idx]
X_test, y_test = X[test_idx], y[test_idx]

X_train = feature_extraction(X_train)
X_test = feature_extraction(X_test)

X_train = X_train.reshape(-1,1001,1)
X_test = X_test.reshape(-1,1001,1)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


# Create Model
model = Sequential()

model.add(
    Conv1D(
        kernel_size=10,
        filters=64,
        padding='same',
        input_shape=(1001,1)))
model.add(BatchNormalization(momentum=0.1, epsilon=0.0001))
model.add(Activation('relu'))
model.add(Conv1D(
        kernel_size=10, filters=128, padding='same'))
model.add(BatchNormalization(momentum=0.1, epsilon=0.0001))
model.add(Activation('relu'))

model.add(Conv1D(
        kernel_size=10, filters=128, padding='same'))
model.add(BatchNormalization(momentum=0.1, epsilon=0.0001))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(12, use_bias=False))
model.add(BatchNormalization(momentum=0.1, epsilon=0.0001))

# Train
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss=squared_hinge, optimizer=adam, metrics=['accuracy'])

model.summary()
model.fit(
    X_train, y_train, batch_size=50, epochs=100, validation_data=(X_test,y_test),
    callbacks=callbacks)
model.evaluate(X_test,y_test)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1001, 64)          64064     
_________________________________________________________________
batch_normalization_1 (Batch (None, 1001, 64)          256       
_________________________________________________________________
activation_1 (Activation)    (None, 1001, 64)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1001, 128)         8192128   
_________________________________________________________________
batch_normalization_2 (Batch (None, 1001, 128)         512       
_________________________________________________________________
activation_2 (Activation)    (None, 1001, 128)         0         
_________________________________________________________________
conv

2300/2300 [==============================] - 105s 46ms/step - loss: 0.9168 - acc: 0.4757 - val_loss: 0.9167 - val_acc: 0.5412
Epoch 38/100
2300/2300 [==============================] - 105s 46ms/step - loss: 0.9168 - acc: 0.4704 - val_loss: 0.9167 - val_acc: 0.5647
Epoch 39/100
2300/2300 [==============================] - 105s 46ms/step - loss: 0.9168 - acc: 0.4674 - val_loss: 0.9167 - val_acc: 0.5922
Epoch 40/100
2300/2300 [==============================] - 105s 46ms/step - loss: 0.9169 - acc: 0.4683 - val_loss: 0.9167 - val_acc: 0.4471
Epoch 41/100
2300/2300 [==============================] - 105s 46ms/step - loss: 0.9168 - acc: 0.4574 - val_loss: 0.9167 - val_acc: 0.5333

Epoch 00041: ReduceLROnPlateau reducing learning rate to 9.999999092680235e-13.
Epoch 42/100
2300/2300 [==============================] - 105s 45ms/step - loss: 0.9170 - acc: 0.4570 - val_loss: 0.9167 - val_acc: 0.6275
Epoch 43/100
2300/2300 [==============================] - 105s 46ms/step - loss: 0.9169 - acc: 0.4

KeyboardInterrupt: 